# Enfrentando o Dragão
**Mural de Quests 04 - Jardim do Palácio**

**Disciplina:** Aprendizado de Máquina

**Professor:** Daniel R. Cassar

**Alunas:** Lília Gavazza, Lívia Aragão, Sophia Alves

Esse é o primeiro teste dos cinco modelos a serem aplicados no dataset escolhido.

In [1]:
# Importando bibliotecas e funções necessárias
import pandas as pd
import numpy as np
import matplotlib as plt
import os 
import shap 

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from optuna import create_study
from sklearn.model_selection import cross_val_score

# [Modelos]
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

C:\Users\lilia25019\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Carregando dados

In [24]:
colunas = ["Crystal System", "Sites", "Energy Above Hull", 
           "Formation Energy", "Volume", "Density", "Band Gap", "Is Gap Direct", "Is Metal",
           "Magnetic Ordering"]

nome_do_arquivo = "magnetismo_Co(in).csv"
magnetic_co = pd.read_csv(nome_do_arquivo)

magnetic_co = magnetic_co.dropna() # retirar linhas com valores vazios 
magnetic_co = magnetic_co.reindex(colunas, axis = 1)

# aplica o conversor LabelEncoder apenas no target "Magnetic Ordering"
label_encoder = LabelEncoder()
magnetic_co["Magnetic Ordering"] = label_encoder.fit_transform(magnetic_co["Magnetic Ordering"])

print("Mapeamento do Label Encoder:")
for i, classe in enumerate(label_encoder.classes_):
    print(f"  {classe} -> {i}")

TAMANHO_TESTE = 0.1 
SEMENTE_ALEATORIA = 42

indices = magnetic_co.index
indices_treino, indices_teste = train_test_split(
    indices, test_size = TAMANHO_TESTE, random_state = SEMENTE_ALEATORIA)

# Definição dos DataFrames com os dados de treino e de teste
treino_mco = magnetic_co.loc[indices_treino]
teste_mco = magnetic_co.loc[indices_teste]


FEATURES = ["Crystal System", "Sites", "Energy Above Hull", 
           "Formation Energy", "Volume", "Density", "Band Gap", "Is Gap Direct", "Is Metal"]
TARGET = ["Magnetic Ordering"]


# Criar DataFrames completos com as FEATURES
X_treino_df = treino_mco.reindex(FEATURES, axis = 1)
X_teste_df = teste_mco.reindex(FEATURES, axis = 1)

# Identificar colunas categóricas
categorical_cols = X_treino_df.select_dtypes(include=['object', 'bool', 'category']).columns.tolist()
numerical_cols = X_treino_df.select_dtypes(include=[np.number]).columns.tolist()

print(f"Colunas categóricas encontradas: {categorical_cols}")

# Criar pré-processador
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ])

# Aplicar transformação aos dados de treino e teste
X_treino_processed = preprocessador.fit_transform(X_treino_df)
X_teste_processed = preprocessador.transform(X_teste_df)

print(f"Shape treino antes: {X_treino_df.shape}, depois: {X_treino_processed.shape}")
print(f"Shape teste antes: {X_teste_df.shape}, depois: {X_teste_processed.shape}")

X_treino = X_treino_processed
X_teste = X_teste_processed

print(X_teste_processed)

# Separação e seleção dos valores de features e target por DF
y_treino = treino_mco.reindex(TARGET, axis = 1).values.ravel() # deixa os dados em 1D - maior precisão
y_teste = teste_mco.reindex(TARGET, axis = 1).values.ravel() # deixa os dados em 1D - maior precisão

Mapeamento do Label Encoder:
  AFM -> 0
  FM -> 1
  FiM -> 2
  NM -> 3
Colunas categóricas encontradas: ['Crystal System', 'Is Gap Direct', 'Is Metal']
Shape treino antes: (242, 9), depois: (242, 16)
Shape teste antes: (27, 9), depois: (27, 16)
[[-5.41704353e-01 -1.41893924e-01 -4.76440251e+00 -7.21368865e-01
  -1.47573379e+00 -1.23738465e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 8.03696067e-01 -2.77112968e-01 -5.27587365e-03  4.52409696e-01
   1.36747056e-01 -1.23738465e-01  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-5.41704353e-01  2.62701109e+00  2.28772513e+00  2.17704890e-02
  -2.06152795e+00 -1.23738465e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.000000

#### Baseline 
Esse é o modelo base, ele apenas prevê a classe mais frequente para todos os dados. Não existe necessidade, portanto de avaliar o melhor conjunto de hiperparâmetro, pois eles não existem nesse caso.

In [25]:
pipeline_bsl = make_pipeline(
    StandardScaler(),
    DummyClassifier(strategy='stratified', random_state=SEMENTE_ALEATORIA)
)
    
pipeline_bsl.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto1 = pipeline_bsl.predict(X_teste)

acuracia_bsl = accuracy_score(y_verdadeiro, y_previsto1)
print(f"Acurácia - Baseline: {round((acuracia_bsl)*100, 2)}%")

Acurácia - Baseline: 37.04%


Esse desempenho será o nosso norte quanto a real capacidade de generalização dos próximos algoritmos, pensando que quanto maior e mais distante dessa base, melhor é o modelo induzido.

### Otimização de Hiperparâmetros com `optuna`
O `optuna` é um módulo de python que permite uma otimização de hiperparâmetros por uma espécie de "conjunto de tentativas". O `trial` é um objeto dele que sorteia e sugere parâmetros para o modelo, criando seu espaço de busca de hiperparâmetros. 

Disso, é definida a função objetivo para medir, reforçada por uma validação cruzada k-fold, a métrica sob análise, nesse caso, a acurácia. Efetivamente o que ocorre é que o conjunto de hiperparâmetros sugerido no passo anterior é avaliado por essa estratégia, que devolve a estimativa de desempenho dele para guiar e comparar com as próximas escolhas. Isso é feito por uma subdivisão dos dados de treino em $k$ subconjuntos, que serão utilizados como dados de teste uma vez. A média das $k$ estimativas de desempenho obtidas é mais robusta em determinar a performance do modelo "induzido".

Por fim, a criação do objeto de estudo define o otimizador desse problema de maximização que será rodado com módulo `optimize` um determinado número de tentativas (100). Os melhores parâmetros encontrados serão os utilizados para realmente induzir o modelo e calcular sua acurácia

#### k-NN Classificador
Esse modelo se baseia na moda de classes de n dos seus vizinhos para prever um dado desconhecido. Seus principais hiperparâmetros são:
- `n_neighbors`: o número de vizinhos a serem considerados na análise da moda;
- `weights`: como as classes dos vizinhos influenciam na previsão, de forma uniforme com todos tendo o meso peso ou de forma guiada pela distância, os mais próximos tem maior peso;
- `p`: o tipo de distância considerada na determinação dos vizinhos mais próximos.

Também será analisada a influência/necessidade da normalização dos dados.

In [4]:
def instancia_kNN(trial):
    parametros_kNN = {
        "n_neighbors": trial.suggest_int("num_vizinhos", 1, 100),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]),
        "p": trial.suggest_float("potencia_minkowski", 1, 2),
        "n_jobs": -1 # processamento
    }
    
    modelo_kNN = KNeighborsClassifier(**parametros_kNN)

    return modelo_kNN           

def func_obj_kNN(trial, X, y, n_folds):
    modelo = instancia_kNN(trial)

    metricas_kNN = cross_val_score(
        modelo,
        X,
        y,
        scoring="f1_weighted",
        cv=n_folds,
        error_score='raise'
    )

    return metricas_kNN.mean()

NOME_DO_ESTUDO = "kNN"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:33:01,723] A new study created in RDB with name: kNN


In [5]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_kNN(trial):
    return func_obj_kNN(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_kNN, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:33:06,462] Trial 0 finished with value: 0.29068670914314365 and parameters: {'num_vizinhos': 87, 'pesos': 'uniform', 'potencia_minkowski': 1.1684473850572037}. Best is trial 0 with value: 0.29068670914314365.
[I 2025-11-03 18:33:06,706] Trial 1 finished with value: 0.4487171731203329 and parameters: {'num_vizinhos': 39, 'pesos': 'uniform', 'potencia_minkowski': 1.4291443745311785}. Best is trial 1 with value: 0.4487171731203329.
[I 2025-11-03 18:33:07,005] Trial 2 finished with value: 0.5745628465914819 and parameters: {'num_vizinhos': 9, 'pesos': 'distance', 'potencia_minkowski': 1.8271925451228714}. Best is trial 2 with value: 0.5745628465914819.
[I 2025-11-03 18:33:07,241] Trial 3 finished with value: 0.3309270750644697 and parameters: {'num_vizinhos': 80, 'pesos': 'uniform', 'potencia_minkowski': 1.7185459068600841}. Best is trial 2 with value: 0.5745628465914819.
[I 2025-11-03 18:33:07,532] Trial 4 finished with value: 0.5426134790545828 and parameters: {'num_vizi

In [6]:
melhor_trial_kNN = objeto_de_estudo.best_trial

modelo_kNN = instancia_kNN(melhor_trial_kNN)
modelo_kNN.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto2 = modelo_kNN.predict(X_teste)

f1_kNN = f1_score(y_verdadeiro, y_previsto2, average = "weighted")

print(f"Melhor trial: {melhor_trial_kNN.number} - Parâmetros: {melhor_trial_kNN.params}")
print(f"Acurácia - kNN: {f1_kNN}")

Melhor trial: 41 - Parâmetros: {'num_vizinhos': 6, 'pesos': 'distance', 'potencia_minkowski': 1.9576388581186748}
Acurácia - kNN: 0.644349477682811


#### Árvore de Decisão
As previsões são feitas por meio da sudivisão dos dados de acordo com limiares de um determinado critério. Seus principais hiperparâmetros são:
- `max_leaf_nodes`: número máximo de vértices folha;
- `max_depth`: máxima profundidade da árvore de decisão gerada;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha.

Novamente, a possibilidade de normalização foi considerada.

In [7]:
def instancia_ad(trial):
    parametros_ad = {
        "max_leaf_nodes": trial.suggest_int("num_maximo_nos", 2, 20),
        "max_depth": trial.suggest_categorical("profundidade_maxima", [None, 3, 6, 9, 12, 18, 21, 24]),
        "min_samples_split": trial.suggest_int("min_exemplos_vcondicional", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_exemplos_vfolha", 1, 20),
        "random_state": 73 #reprotibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_ad = make_pipeline(
            StandardScaler(),
            DecisionTreeClassifier(**parametros_ad)
        )

    else:
        modelo_ad = DecisionTreeClassifier(**parametros_ad)

    return modelo_ad           

def func_obj_ad(trial, X, y, n_folds):
    modelo = instancia_ad(trial)

    metricas_ad = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        
        cv=n_folds
    )

    return metricas_ad.mean()

NOME_DO_ESTUDO = "arvore_de_decisao"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:33:36,414] A new study created in RDB with name: arvore_de_decisao


In [8]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_ad(trial):
    return func_obj_ad(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_ad, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:33:36,730] Trial 0 finished with value: 0.528061224489796 and parameters: {'num_maximo_nos': 10, 'profundidade_maxima': 12, 'min_exemplos_vcondicional': 14, 'min_exemplos_vfolha': 13, 'normalizacao': True}. Best is trial 0 with value: 0.528061224489796.
[I 2025-11-03 18:33:37,054] Trial 1 finished with value: 0.5409013605442177 and parameters: {'num_maximo_nos': 5, 'profundidade_maxima': 6, 'min_exemplos_vcondicional': 9, 'min_exemplos_vfolha': 11, 'normalizacao': True}. Best is trial 1 with value: 0.5409013605442177.
[I 2025-11-03 18:33:37,324] Trial 2 finished with value: 0.536734693877551 and parameters: {'num_maximo_nos': 11, 'profundidade_maxima': 9, 'min_exemplos_vcondicional': 19, 'min_exemplos_vfolha': 7, 'normalizacao': False}. Best is trial 1 with value: 0.5409013605442177.
[I 2025-11-03 18:33:37,572] Trial 3 finished with value: 0.5449829931972789 and parameters: {'num_maximo_nos': 8, 'profundidade_maxima': None, 'min_exemplos_vcondicional': 8, 'min_exemplos

In [9]:
melhor_trial_ad = objeto_de_estudo.best_trial

modelo_ad = instancia_ad(melhor_trial_ad)
modelo_ad.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto3 = modelo_ad.predict(X_teste)

acuracia_ad = accuracy_score(y_verdadeiro, y_previsto3)

print(f"Melhor trial: {melhor_trial_ad.number} - Parâmetros: {melhor_trial_ad.params}")
print(f"Acurácia - Árvore de Decisão: {acuracia_ad}")

Melhor trial: 52 - Parâmetros: {'num_maximo_nos': 9, 'profundidade_maxima': 21, 'min_exemplos_vcondicional': 4, 'min_exemplos_vfolha': 4, 'normalizacao': False}
Acurácia - Árvore de Decisão: 0.5555555555555556


#### Floresta Aleatória
Nesse algoritmos são projetadas vários conjuntos de árvores de decisão, tentando criar um modelo de maior variância e menor viés. Seus principais hiperparâmetros são:
- `n_estimators`: número de árvores por comitê;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha;
- `max_features`: fração de atributos usada em cada _split_ das árvores;
- `bootstrap`: se ocorre uma amostragem do tipo _bootstrap_ dos exemplos para treinar as árvores.

In [10]:
def instancia_fa(trial):
    parametros_fa = {
        "n_estimators": trial.suggest_int("num_arvores", 50, 500, log=True),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 10),
        "max_features": trial.suggest_float("num_max_atributos", 0, 1),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "n_jobs": -1, #processamento
        "random_state": 73 #reprodutibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])
    
    if normalizar == True:
        modelo_fa = make_pipeline(
            StandardScaler(),
            RandomForestClassifier(**parametros_fa)
        )

    else:
        modelo_fa= RandomForestClassifier(**parametros_fa)

    return modelo_fa

def func_obj_fa(trial, X, y, num_folds):
    modelo = instancia_fa(trial)

    metricas_fa = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_fa.mean()

NOME_DO_ESTUDO = "floresta_aleatoria"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:34:05,341] A new study created in RDB with name: floresta_aleatoria


In [11]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_fa(trial):
    return func_obj_fa(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_fa, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:34:09,485] Trial 0 finished with value: 0.5821428571428571 and parameters: {'num_arvores': 285, 'min_exemplos_split': 3, 'min_exemplos_folha': 4, 'num_max_atributos': 0.36783880116402246, 'bootstrap': True, 'normalizacao': True}. Best is trial 0 with value: 0.5821428571428571.
[I 2025-11-03 18:34:11,192] Trial 1 finished with value: 0.5988095238095238 and parameters: {'num_arvores': 89, 'min_exemplos_split': 2, 'min_exemplos_folha': 6, 'num_max_atributos': 0.2294347609194609, 'bootstrap': True, 'normalizacao': False}. Best is trial 1 with value: 0.5988095238095238.
[I 2025-11-03 18:34:13,528] Trial 2 finished with value: 0.5695578231292517 and parameters: {'num_arvores': 143, 'min_exemplos_split': 7, 'min_exemplos_folha': 4, 'num_max_atributos': 0.8628451233586915, 'bootstrap': True, 'normalizacao': True}. Best is trial 1 with value: 0.5988095238095238.
[I 2025-11-03 18:34:15,591] Trial 3 finished with value: 0.5947278911564626 and parameters: {'num_arvores': 129, 'min

In [12]:
melhor_trial_fa = objeto_de_estudo.best_trial

modelo_fa = instancia_fa(melhor_trial_fa)
modelo_fa.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto4 = modelo_fa.predict(X_teste)

acuracia_fa = accuracy_score(y_verdadeiro, y_previsto4)

print(f"Melhor trial: {melhor_trial_fa.number} - Parâmetros: {melhor_trial_fa.params}")
print(f"Acurácia - Floresta Aleatória: {acuracia_fa:.2f}")

Melhor trial: 69 - Parâmetros: {'num_arvores': 360, 'min_exemplos_split': 3, 'min_exemplos_folha': 4, 'num_max_atributos': 0.18207512667856973, 'bootstrap': True, 'normalizacao': True}
Acurácia - Floresta Aleatória: 0.67


#### Naive Bayes Gaussiano
Esse algoritmo assume _features_ independentes e faz previsões baseadas nas probailidades das classes assumindo as _features_ em distribuição gaussiana e o Teorema de Bayes. Seu principal hiperparâmetro é:
- `var_smoothing`: controla a variância que afeta as distribuições das _features_, para aumentar a estabilidade da previsão.

In [13]:
def instancia_nbg(trial):
    parametros_nbg = {
        "var_smoothing": trial.suggest_float("var_smoothing", 1e-11, 1e-1, log=True)
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])
    
    if normalizar == True:
        modelo_nbg= make_pipeline(
            StandardScaler(),
            GaussianNB(**parametros_nbg)
        )

    else:
        modelo_nbg= GaussianNB(**parametros_nbg)
    
    return modelo_nbg

def func_obj_nbg(trial, X, y, num_folds):
    modelo = instancia_nbg(trial)

    metricas_nbg = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_nbg.mean()

NOME_DO_ESTUDO = "naive_bayes_gaussiano"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:40:42,929] A new study created in RDB with name: naive_bayes_gaussiano


In [14]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_nbg(trial):
    return func_obj_nbg(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_nbg, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:40:43,097] Trial 0 finished with value: 0.5451530612244898 and parameters: {'var_smoothing': 0.0023475820301751197, 'normalizacao': False}. Best is trial 0 with value: 0.5451530612244898.
[I 2025-11-03 18:40:43,235] Trial 1 finished with value: 0.5783163265306123 and parameters: {'var_smoothing': 0.01115309669837026, 'normalizacao': False}. Best is trial 1 with value: 0.5783163265306123.
[I 2025-11-03 18:40:43,385] Trial 2 finished with value: 0.45858843537414967 and parameters: {'var_smoothing': 2.5197835955971552e-05, 'normalizacao': True}. Best is trial 1 with value: 0.5783163265306123.
[I 2025-11-03 18:40:43,530] Trial 3 finished with value: 0.46700680272108847 and parameters: {'var_smoothing': 9.457386647992648e-11, 'normalizacao': True}. Best is trial 1 with value: 0.5783163265306123.
[I 2025-11-03 18:40:43,666] Trial 4 finished with value: 0.4710884353741497 and parameters: {'var_smoothing': 5.694525137437477e-10, 'normalizacao': False}. Best is trial 1 with val

In [15]:
melhor_trial_nbg = objeto_de_estudo.best_trial

modelo_nbg = instancia_nbg(melhor_trial_nbg)
modelo_nbg.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto5 = modelo_nbg.predict(X_teste)

acuracia_nbg = accuracy_score(y_verdadeiro, y_previsto5)

print(f"Melhor trial: {melhor_trial_nbg.number} - Parâmetros: {melhor_trial_nbg.params}")
print(f"Acurácia - Naive Bayes Gaussiano: {acuracia_nbg:.2f}")

Melhor trial: 12 - Parâmetros: {'var_smoothing': 0.0757065689985383, 'normalizacao': False}
Acurácia - Naive Bayes Gaussiano: 0.59


#### Support Vector Classification - SVC
Nesse modelo, a multiclassficação é subdividida em problemas de classificação binária. Em cada um deles, procura o hiperplano que melhora separa as diferentes classes. Seus principais hiperparâmetros são:
- `kernel`: método de separação dos grupos;
- `C`: parâmetro de regularização;
- `gamma`: controla a influência de cada ponto de treino em _kernels_ não lineares;
- `max_iter`:

In [16]:
def instancia_svc(trial):
    parametros_svc = {
        "kernel": trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"]),
        "C": trial.suggest_float("C", 0.01, 100, log=True),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "max_iter": 5000,
        "random_state": 73 #reprodutibilidade
    }

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_svc = make_pipeline(
            StandardScaler(),
            SVC(**parametros_svc)
        )

    else:
        modelo_svc = SVC(**parametros_svc)

    return modelo_svc  



def func_obj_svc(trial, X, y, num_folds):
    modelo = instancia_svc(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "svc"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:40:59,247] A new study created in RDB with name: svc


In [17]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_svc(trial):
    return func_obj_svc(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_svc, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:40:59,539] Trial 0 finished with value: 0.400765306122449 and parameters: {'kernel': 'rbf', 'C': 0.013172237786140026, 'gamma': 'auto', 'normalizacao': False}. Best is trial 0 with value: 0.400765306122449.
[I 2025-11-03 18:40:59,784] Trial 1 finished with value: 0.400765306122449 and parameters: {'kernel': 'poly', 'C': 0.018677274850948992, 'gamma': 'auto', 'normalizacao': False}. Best is trial 0 with value: 0.400765306122449.
[I 2025-11-03 18:41:00,018] Trial 2 finished with value: 0.5782312925170068 and parameters: {'kernel': 'rbf', 'C': 22.75146247892017, 'gamma': 'auto', 'normalizacao': False}. Best is trial 2 with value: 0.5782312925170068.
[I 2025-11-03 18:41:00,234] Trial 3 finished with value: 0.4337585034013606 and parameters: {'kernel': 'rbf', 'C': 0.1633380715979603, 'gamma': 'auto', 'normalizacao': False}. Best is trial 2 with value: 0.5782312925170068.
[I 2025-11-03 18:41:00,464] Trial 4 finished with value: 0.549234693877551 and parameters: {'kernel': 's

In [18]:
melhor_trial_svc = objeto_de_estudo.best_trial

modelo_svc = instancia_svc(melhor_trial_svc)
modelo_svc.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_svc.predict(X_teste)

acuracia_svc = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_svc.number} - Parâmetros: {melhor_trial_svc.params}")
print(f"Acurácia - SVC: {acuracia_svc}")

Melhor trial: 34 - Parâmetros: {'kernel': 'rbf', 'C': 8.134234798526416, 'gamma': 'auto', 'normalizacao': False}
Acurácia - SVC: 0.5555555555555556


#### Regressão Logística

In [19]:
def instancia_lr(trial):
    parametros_lr = {
        'penalty':  trial.suggest_categorical("penalty", ['l1','l2']),
        'C': trial.suggest_float("C", 1e-4, 1e4, log=True),
        'solver': "saga",
        'max_iter': trial.suggest_int("max_iter", 100, 5000, log=True),
        'tol': 1e-2,
        'random_state': 73
}

    normalizar = trial.suggest_categorical("normalizacao", [False, True])

    if normalizar == True:
        modelo_lr = make_pipeline(
            StandardScaler(),
            LogisticRegression(**parametros_lr)
        )

    else:
        modelo_lr = LogisticRegression(**parametros_lr)

    return modelo_lr


def func_obj_lr(trial, X, y, num_folds):
    modelo = instancia_lr(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "lr"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 18:41:23,031] A new study created in RDB with name: lr


In [20]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_lr(trial):
    return func_obj_lr(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_lr, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 18:41:23,343] Trial 0 finished with value: 0.400765306122449 and parameters: {'penalty': 'l2', 'C': 0.0007969562502491485, 'max_iter': 178, 'normalizacao': False}. Best is trial 0 with value: 0.400765306122449.
[I 2025-11-03 18:41:23,555] Trial 1 finished with value: 0.421343537414966 and parameters: {'penalty': 'l2', 'C': 0.0036649693573591797, 'max_iter': 3516, 'normalizacao': False}. Best is trial 1 with value: 0.421343537414966.
[I 2025-11-03 18:41:23,819] Trial 2 finished with value: 0.5988945578231293 and parameters: {'penalty': 'l2', 'C': 1112.1265511759198, 'max_iter': 3716, 'normalizacao': False}. Best is trial 2 with value: 0.5988945578231293.
[I 2025-11-03 18:41:24,062] Trial 3 finished with value: 0.5988945578231293 and parameters: {'penalty': 'l2', 'C': 1302.5145227516962, 'max_iter': 3543, 'normalizacao': True}. Best is trial 2 with value: 0.5988945578231293.
[I 2025-11-03 18:41:24,296] Trial 4 finished with value: 0.5947278911564625 and parameters: {'penalt

In [23]:
melhor_trial_lr = objeto_de_estudo.best_trial

modelo_lr = instancia_lr(melhor_trial_lr)
modelo_lr.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_lr.predict(X_teste)

acuracia_lr = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_lr.number} - Parâmetros: {melhor_trial_lr.params}")
print(f"Acurácia - SVC: {acuracia_lr}")

Melhor trial: 24 - Parâmetros: {'penalty': 'l1', 'C': 2.220418325077822, 'max_iter': 456, 'normalizacao': False}
Acurácia - SVC: 0.6296296296296297


### Referências
1. `sckit-learn` - Geração de Datasets. Disponível em: [https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators](https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators)
2. `sckit-learn` - Validação Cruzada. Disponível em: [https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)
3. `sckit-learn` - Naive Bayes. Disponível em: [https://scikit-learn.org/stable/modules/naive_bayes.html](https://scikit-learn.org/stable/modules/naive_bayes.html)
4. BAELDUNG. Multiclass Classification Using Support Vector Machines | Baeldung on Computer Science. Disponível em: [https://www.baeldung.com/cs/svm-multiclass-classification](https://www.baeldung.com/cs/svm-multiclass-classification)
5. `sckit-learn` - `SVC`. Disponível em: [https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 
6. CASSAR, Daniel. Materiais de Aula.